In [ ]:
!git clone https://github.com/nuttachot/ai101.git

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
df = pd.read_csv('THB.csv')

In [ ]:
df.head()

In [ ]:
df.columns.values

In [ ]:
df.shape

In [ ]:
df['Date']

In [ ]:
df['Close'].describe()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df['Close'], label='Close Price history')

In [ ]:
df.hist(bins=15, color='steelblue', edgecolor='black', linewidth=1.0, xlabelsize=8, ylabelsize=8, grid=False)    
plt.tight_layout(rect=(0, 0, 1.2, 1.2))   

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
boxplot = df.boxplot(column=['Close', 'Open', 'High', 'Low'])

In [ ]:
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

for i in range(0,len(data)):
     new_data['Date'][i] = data['Date'][i]
     new_data['Close'][i] = data['Close'][i]

In [ ]:
cut = 1292

In [ ]:
train = new_data[:cut]
valid = new_data[cut:]

In [ ]:
train.shape, valid.shape

In [ ]:
train['Date'].min(), train['Date'].max(), valid['Date'].min(), valid['Date'].max()

In [ ]:
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

dataset = new_data.values
train = dataset[0:cut,:]
valid = dataset[cut:,:]
new_data.shape, train.shape, valid.shape

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
#print(dataset[0:60,0])

scaled_data = scaler.fit_transform(dataset)
#print(scaled_data[0:60,0])

In [ ]:
x_train, y_train = [], []
for i in range(3,len(train)):
    x_train.append(scaled_data[i-3:i,0])
    y = []
    y.append(scaled_data[i,0])
   
    y_train.append(y)

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train.shape

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout

In [ ]:
# define model

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=3))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
# fit model
model.fit(x_train, y_train, epochs=10, verbose=1, batch_size=1, shuffle=True)

In [ ]:
inputs = new_data[len(new_data) - len(valid) - 3:].values
inputs  = scaler.transform(inputs)
inputs.shape

In [ ]:
X_test = []
for i in range(3,inputs.shape[0]):
    X_test.append(inputs[i-3:i,0])
X_test = np.array(X_test)

In [ ]:
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
train = new_data[:cut]
valid = new_data[cut:]
valid['Predictions'] = closing_price

plt.figure(figsize=(16,8))
plt.plot(train['Close'],)
plt.plot(valid[['Close','Predictions']])